<a href="https://colab.research.google.com/github/mayankcircle/Tensorflow-2.0/blob/main/Tensorflow_Tutorial_5_(Regularization).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [20]:
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers, regularizers
from tensorflow.keras.datasets import cifar10

In [2]:
tf.__version__

'2.3.0'

# Load CIFAR10 Dataset

In [3]:
(X_train, y_train),(X_test,y_test) = cifar10.load_data()

170500096/170498071 [==============================] - 4s 0us/step


In [4]:
X_train.shape

(50000, 32, 32, 3)

In [5]:
set(y_train.ravel())

{0, 1, 2, 3, 4, 5, 6, 7, 8, 9}

**CIFAR10 train dataset has 50000 RGB images of 32 x 32 px of 10 classes.**

We convert each value to flaot32 dtype and normalize for fast training.

In [6]:
X_train = X_train.astype("float32") / 255.0
X_test = X_test.astype("float32") / 255.0

# Basic CNN Model using Sequential API

In [7]:
model = keras.Sequential(
    [
     keras.Input(shape=(32,32,3)), # h,w,channels (Shape of 1 image). By default, data_format is "channels_last"
     # 1st convolutional layer : we use 32 filters of size 3 x 3 with No padding (valid)
     layers.Conv2D(filters=32,kernel_size=3,padding="valid",activation="relu",name="1st_CONV"),
     # max pool layer
     layers.MaxPool2D(pool_size=(2,2),name="1st_POOL"),
     # 2nd convolutional layer : we use 64 filters of size 3 x 3 with No padding (valid)
     # By default, padding is "valid"
     layers.Conv2D(filters=64,kernel_size=3,activation="relu",name="2nd_CONV"),
     # max pool layer
     # By default pool_size = (2,2)
     layers.MaxPool2D(name="2nd_POOL"),
     # 3rd convolutional layer : we use 128 filters of size 3 x 3 with No padding (valid)
     layers.Conv2D(filters=128,kernel_size=3,activation="relu", name="3rd_CONV"),
     # flatten layer
     layers.Flatten(),

     # 1st FC layer
     layers.Dense(64,name="FC1"),
     # Output layer # 10 classes : logits
     layers.Dense(10,name="FC2")

    ]
)

In [8]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
1st_CONV (Conv2D)            (None, 30, 30, 32)        896       
_________________________________________________________________
1st_POOL (MaxPooling2D)      (None, 15, 15, 32)        0         
_________________________________________________________________
2nd_CONV (Conv2D)            (None, 13, 13, 64)        18496     
_________________________________________________________________
2nd_POOL (MaxPooling2D)      (None, 6, 6, 64)          0         
_________________________________________________________________
3rd_CONV (Conv2D)            (None, 4, 4, 128)         73856     
_________________________________________________________________
flatten (Flatten)            (None, 2048)              0         
_________________________________________________________________
FC1 (Dense)                  (None, 64)                1

In [9]:
model.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=keras.optimizers.Adam(lr=3e-4),
    metrics=["accuracy"]
)

In [10]:
# train
model.fit(X_train,y_train,batch_size=64,epochs=2)

Epoch 1/2
782/782 [==============================] - 3s 3ms/step - loss: 1.6156 - accuracy: 0.4156
Epoch 2/2
782/782 [==============================] - 3s 3ms/step - loss: 1.2876 - accuracy: 0.5427


In [11]:
model.evaluate(X_test,y_test,batch_size=64)

157/157 [==============================] - 0s 2ms/step - loss: 1.1962 - accuracy: 0.5813


[1.196179747581482, 0.5813000202178955]

# Basic CNN model using Functional API without Regularization

Build simple basic CNN model without any regularization technique

In [12]:
def my_model_without_regularization():
    inputs = keras.Input(shape=(32,32,3))
    x = layers.Conv2D(filters=32,kernel_size=3,name="1st_CONV")(inputs)
    x = layers.BatchNormalization()(x) # we should not applying activation function before Batch Normalization, we need CONV layer RAW Output directly here
    x = keras.activations.relu(x) # apply activation after Batch Norm
    x = layers.MaxPool2D()(x) # by default, pooling window is 2 x 2
    x = layers.Conv2D(filters=64,kernel_size=5,padding="same",name="2nd_CONV")(x)
    x = layers.BatchNormalization()(x)
    x = keras.activations.relu(x)
    x = layers.Conv2D(filters=128,kernel_size=3,name="3rd_CONV")(x)
    x = layers.BatchNormalization()(x)
    x = keras.activations.relu(x)
    x = layers.Flatten()(x)
    x = layers.Dense(64,activation="relu")(x)
    outputs = layers.Dense(10)(x)
    
    # create model
    model = keras.Model(inputs=inputs, outputs = outputs)

    return model


In [13]:
model = my_model_without_regularization()

In [14]:
model.summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 32, 32, 3)]       0         
_________________________________________________________________
1st_CONV (Conv2D)            (None, 30, 30, 32)        896       
_________________________________________________________________
batch_normalization (BatchNo (None, 30, 30, 32)        128       
_________________________________________________________________
tf_op_layer_Relu (TensorFlow [(None, 30, 30, 32)]      0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 15, 15, 32)        0         
_________________________________________________________________
2nd_CONV (Conv2D)            (None, 15, 15, 64)        51264     
_________________________________________________________________
batch_normalization_1 (Batch (None, 15, 15, 64)       

In [15]:
model.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=keras.optimizers.Adam(lr=3e-4),
    metrics=["accuracy"]
)

In [18]:
# train
model.fit(X_train,y_train,batch_size=64,epochs=10)

Epoch 1/10
782/782 [==============================] - 5s 6ms/step - loss: 0.7268 - accuracy: 0.7487
Epoch 2/10
782/782 [==============================] - 5s 6ms/step - loss: 0.6228 - accuracy: 0.7831
Epoch 3/10
782/782 [==============================] - 5s 6ms/step - loss: 0.5295 - accuracy: 0.8145
Epoch 4/10
782/782 [==============================] - 5s 6ms/step - loss: 0.4485 - accuracy: 0.8474
Epoch 5/10
782/782 [==============================] - 5s 6ms/step - loss: 0.3785 - accuracy: 0.8704
Epoch 6/10
782/782 [==============================] - 5s 6ms/step - loss: 0.3124 - accuracy: 0.8947
Epoch 7/10
782/782 [==============================] - 5s 6ms/step - loss: 0.2595 - accuracy: 0.9121
Epoch 8/10
782/782 [==============================] - 5s 6ms/step - loss: 0.2084 - accuracy: 0.9315
Epoch 9/10
782/782 [==============================] - 5s 6ms/step - loss: 0.1732 - accuracy: 0.9447
Epoch 10/10
782/782 [==============================] - 5s 6ms/step - loss: 0.1368 - accuracy: 0.9575

In [19]:
model.evaluate(X_test,y_test,batch_size=64)

157/157 [==============================] - 0s 3ms/step - loss: 1.0464 - accuracy: 0.7308


[1.0463511943817139, 0.7307999730110168]

The gap between training and test accuracy is large, it means it is overfitted on training data. We can solve this using **REgularization**.

# Basic CNN model using Functional API with Regularization

we can add regularizer seperately in layers. Here, we are using L2 Regularizer.

we are also using dropout in FC Layers.

As we had already **BatchNormalization**, **the purpose of Batch NOrmalization is to normalize weights to have faster training but it also has some regularization effect**

In [23]:
def my_model_with_regularization():
    inputs = keras.Input(shape=(32,32,3))
    x = layers.Conv2D(filters=32,kernel_size=3,name="1st_CONV",kernel_regularizer=regularizers.l2(0.01))(inputs)
    x = layers.BatchNormalization()(x) # we should not applying activation function before Batch Normalization, we need CONV layer RAW Output directly here
    x = keras.activations.relu(x) # apply activation after Batch Norm
    x = layers.MaxPool2D()(x) # by default, pooling window is 2 x 2
    x = layers.Conv2D(filters=64,kernel_size=5,padding="same",name="2nd_CONV", kernel_regularizer=regularizers.l2(0.01))(x)
    x = layers.BatchNormalization()(x)
    x = keras.activations.relu(x)
    x = layers.Conv2D(filters=128,kernel_size=3,name="3rd_CONV",kernel_regularizer=regularizers.l2(0.01))(x)
    x = layers.BatchNormalization()(x)
    x = keras.activations.relu(x)
    x = layers.Flatten()(x)
    x = layers.Dense(64,activation="relu", kernel_regularizer=regularizers.l2(0.01))(x)
    x = layers.Dropout(0.5)(x) # dropout layer
    outputs = layers.Dense(10)(x)
    
    # create model
    model = keras.Model(inputs=inputs, outputs = outputs)

    return model

In [24]:
model = my_model_with_regularization()

In [25]:
model.summary()

Model: "functional_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         [(None, 32, 32, 3)]       0         
_________________________________________________________________
1st_CONV (Conv2D)            (None, 30, 30, 32)        896       
_________________________________________________________________
batch_normalization_4 (Batch (None, 30, 30, 32)        128       
_________________________________________________________________
tf_op_layer_Relu_4 (TensorFl [(None, 30, 30, 32)]      0         
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 15, 15, 32)        0         
_________________________________________________________________
2nd_CONV (Conv2D)            (None, 15, 15, 64)        51264     
_________________________________________________________________
batch_normalization_5 (Batch (None, 15, 15, 64)       

In [26]:
model.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=keras.optimizers.Adam(lr=3e-4),
    metrics=["accuracy"]
)

Since , we are using Dropout layer, we need to do more epochs in training because we are dropping connections randomly and it needs more iteration to train the network.

In [27]:
# large epochs due to dropout
model.fit(X_train,y_train,batch_size=64,epochs=150)

Epoch 1/150
782/782 [==============================] - 5s 7ms/step - loss: 3.1128 - accuracy: 0.1650
Epoch 2/150
782/782 [==============================] - 5s 7ms/step - loss: 2.1527 - accuracy: 0.2481
Epoch 3/150
782/782 [==============================] - 5s 7ms/step - loss: 1.9350 - accuracy: 0.3058
Epoch 4/150
782/782 [==============================] - 5s 7ms/step - loss: 1.8456 - accuracy: 0.3332
Epoch 5/150
782/782 [==============================] - 5s 7ms/step - loss: 1.7688 - accuracy: 0.3574
Epoch 6/150
782/782 [==============================] - 5s 7ms/step - loss: 1.7373 - accuracy: 0.3860
Epoch 7/150
782/782 [==============================] - 5s 7ms/step - loss: 1.6976 - accuracy: 0.4046
Epoch 8/150
782/782 [==============================] - 5s 7ms/step - loss: 1.6866 - accuracy: 0.4119
Epoch 9/150
782/782 [==============================] - 5s 7ms/step - loss: 1.6675 - accuracy: 0.4203
Epoch 10/150
782/782 [==============================] - 5s 7ms/step - loss: 1.6586 - accura

In [28]:
model.evaluate(X_test,y_test,batch_size=64)

157/157 [==============================] - 0s 3ms/step - loss: 1.2204 - accuracy: 0.6521


[1.220400333404541, 0.6521000266075134]

It is now looking more underfit. MOre learing is required.